<a href="https://colab.research.google.com/github/achirde/machine_learning/blob/master/Bitcoin_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy  as np

from sklearn.ensemble      import GradientBoostingRegressor
from sklearn.linear_model  import LinearRegression
from sklearn.preprocessing import StandardScaler

**Configuration**<br>
Tune the regressor's parameter here. <br>

We set our file names also here.

In [0]:
# GB Parameters
LOSS = 'huber'
CRITERION = 'friedman_mse'
MAX_DEPTH = 32

# Configure Data Options
ENABLE_TWEET = True

# File Names
SUB_FILE_NAME          = 'SUBMISSION_GB_HUBER_FMSE_32.csv'
TRAIN_TIMESERIES       = 'train_timeseries2.csv'
CLEAN_TWEET_TRAIN_FILE = 'clean_tw_data_train_2.csv'
ACTUAL_DATA_FILE       = 'sample_submission_2_actual_values.csv'
TEST_TIMESERIES        = 'test_timeseries2.csv'
CLEAN_TWEET_DATA_TEST  = 'clean_tw_data_test.csv'

**Gradient Boost**
Loss = huber ; Criteria = FMSE, Max Depth = 32 ; Columns = Selected
=> 0.9346

In [0]:
columns = ['close',          
           'cost_per_transaction',
           'est_transaction_volume_USD',
           'est_transaction_volume',
           'cost_per_transaction_percent' ,
           'transaction_fees',
           'n_transactions_per_block',
           'utxo_count_max' ,
           'utxo_count_min' ,
           'mempool_size_max'
            ]

columns = list(set(columns))  #This is just to ensure we have unique column names and no duplicates entered by mistake

In [87]:
# Read Time Series Data
data = pd.read_csv(TRAIN_TIMESERIES, index_col=0)
data = data[columns]
data.head()

,est_transaction_volume_USD,cost_per_transaction_percent,mempool_size_max,est_transaction_volume,n_transactions_per_block,utxo_count_max,transaction_fees,close,cost_per_transaction,utxo_count_min
date,,,,,,,,,,
2016-04-24,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-25,168025613.1,0.888460,8371544.0,362233.4608,1713.102362,36893715,43.298470,461.43,6.861613,36825828
2016-04-26,170332016.5,1.129962,9632225.0,363972.8546,1660.556291,36980350,337.756398,466.09,7.675906,36895435
2016-04-27,167458968.2,0.873767,11592705.0,374368.9346,1856.085271,37044068,46.111500,444.69,6.111057,36983148
2016-04-28,171720826.0,0.904082,12091996.0,381601.8356,1836.725926,37106607,49.620345,449.01,6.261128,37046272


In [88]:
# Read Tweet Data (from clean file)
data_tweets = pd.read_csv(CLEAN_TWEET_TRAIN_FILE)
data_tweets.drop(data_tweets.columns[0], axis=1, inplace=True)

# Remove Account and Favorite Count columns
data_tweets.drop(['account'], axis=1, inplace=True)
data_tweets.drop(['favorite_count'], axis=1, inplace=True)


data_tweets.set_index(['date'], inplace=True)
data_tweets.head()

,retweet_count,follower_count,pol,sub
date,,,,
2016-04-24,111.0,36499.0,0.133333,0.133333
2016-04-24,91.0,36499.0,0.500000,0.500000
2016-04-24,54.0,36499.0,1.000000,1.000000
2016-04-24,0.0,18633.0,0.300000,0.300000
2016-04-24,0.0,18633.0,0.000000,0.000000


In [89]:
data_close = data[columns]
data_w_tweets = data_tweets.merge(data_close, left_index=True, right_index=True)
data_w_tweets.head()

,retweet_count,follower_count,pol,sub,est_transaction_volume_USD,cost_per_transaction_percent,mempool_size_max,est_transaction_volume,n_transactions_per_block,utxo_count_max,transaction_fees,close,cost_per_transaction,utxo_count_min
date,,,,,,,,,,,,,,
2016-04-24,111.0,36499.0,0.133333,0.133333,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,91.0,36499.0,0.500000,0.500000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,54.0,36499.0,1.000000,1.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.300000,0.300000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.000000,0.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621


In [90]:
if ENABLE_TWEET:
  data = data_w_tweets
  
data.head()

,retweet_count,follower_count,pol,sub,est_transaction_volume_USD,cost_per_transaction_percent,mempool_size_max,est_transaction_volume,n_transactions_per_block,utxo_count_max,transaction_fees,close,cost_per_transaction,utxo_count_min
date,,,,,,,,,,,,,,
2016-04-24,111.0,36499.0,0.133333,0.133333,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,91.0,36499.0,0.500000,0.500000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,54.0,36499.0,1.000000,1.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.300000,0.300000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.000000,0.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621


In [91]:
len(data)

53660

In [92]:
data[data.isnull().any(axis=1)]

,retweet_count,follower_count,pol,sub,est_transaction_volume_USD,cost_per_transaction_percent,mempool_size_max,est_transaction_volume,n_transactions_per_block,utxo_count_max,transaction_fees,close,cost_per_transaction,utxo_count_min
date,,,,,,,,,,,,,,
2017-07-28,NaN,NaN,0.0,0.0,1.328334e+09,0.432099,10110016.0,477537.0304,1648.046667,53466489,188.431323,2809.01,23.218259,53429648
2017-09-08,8.0,43361.0,NaN,NaN,1.024261e+09,1.012291,16819637.5,237606.1058,1597.074286,53843463,217.765725,4228.75,37.098193,53795032


In [0]:
data = data.dropna(how='any')

In [94]:
data.head()

,retweet_count,follower_count,pol,sub,est_transaction_volume_USD,cost_per_transaction_percent,mempool_size_max,est_transaction_volume,n_transactions_per_block,utxo_count_max,transaction_fees,close,cost_per_transaction,utxo_count_min
date,,,,,,,,,,,,,,
2016-04-24,111.0,36499.0,0.133333,0.133333,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,91.0,36499.0,0.500000,0.500000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,54.0,36499.0,1.000000,1.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.300000,0.300000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621
2016-04-24,0.0,18633.0,0.000000,0.000000,110486293.2,1.385967,5165259.0,240501.2912,1387.537879,36825214,33.268655,458.55,8.360698,36724621


In [0]:
data_train = data.sample(frac=0.8, random_state=0)
data_test = data.drop(data_train.index)

In [0]:
train_stat = data_train.describe()
train_stat.pop('close')
train_stat = train_stat.transpose()

In [0]:
y_train = data_train.pop("close")
y_test  = data_test.pop("close")

**Normalize the data**

In [98]:
sc = StandardScaler()
normed_X_train = sc.fit_transform(data_train)
normed_X_test = sc.transform(data_test)

# normed_X_train = data_train
# normed_X_test  = data_test

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
reg = GradientBoostingRegressor(loss=LOSS, criterion = CRITERION, max_depth=MAX_DEPTH)
  
reg.fit(normed_X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=32,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [0]:
y_pred = reg.predict(normed_X_test)
y_pred = y_pred.round(decimals=2)
score = reg.score(normed_X_test, y_test)

In [101]:
score

0.8179193822843805

In [0]:
y_pred_df = pd.DataFrame(y_pred, columns=['close'], index=y_test.index)

In [103]:
y_pred_df.head()

,close
date,
2016-10-09,669.09
2016-12-03,780.00
2016-12-03,779.92


**Prepare Submission File**

In [104]:
columns.remove('close')
data_sub = pd.read_csv(TEST_TIMESERIES, index_col=0)
data_sub = data_sub[columns]

data_tweets_sub = pd.read_csv(CLEAN_TWEET_DATA_TEST)
data_tweets_sub.drop(data_tweets_sub.columns[0], axis=1, inplace=True)
data_tweets_sub.drop(['account'], axis=1, inplace=True)
data_tweets_sub.drop(['favorite_count'], axis=1, inplace=True)


data_tweets_sub.set_index(['date'], inplace=True)

data_close_sub = data_sub[columns]
data_w_tweets_sub = data_tweets_sub.merge(data_close_sub, left_index=True, right_index=True)

#Enable Tweet Data
if ENABLE_TWEET:
  data_sub = data_w_tweets_sub  

data_sub = data_sub.dropna(how='any')

normalized_sub = sc.transform(data_sub)
# normalized_sub = data_sub

y_sub = reg.predict(normalized_sub)
y_sub = y_sub.round(decimals=2)
y_sub_df = pd.DataFrame(y_sub, columns=['close'], index=data_sub.index)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [105]:
y_sub_df.head()

,close
date,
2018-12-01,3997.26
2018-12-01,3997.26
2018-12-01,3997.26
2018-12-01,3997.26
2018-12-01,3997.26


In [0]:
y_sub_df = y_sub_df.groupby('date').agg('mean')

**Validate Output file**

In [107]:
data_val = pd.read_csv(ACTUAL_DATA_FILE, index_col=0)
data_val.head()

,close
date,
2018-12-01,4214.67
2018-12-02,4139.88
2018-12-03,3894.13
2018-12-04,3956.89
2018-12-05,3753.99


In [108]:
y_sub_df.columns[0]
y_sub_df.head()

,close
date,
2018-12-01,3997.26
2018-12-02,3982.33
2018-12-03,4212.72
2018-12-04,4014.75
2018-12-05,4082.48


In [0]:
val_data = data_val.merge(y_sub_df, left_index=True, right_index=True)

In [110]:
val_data.head()

,close_x,close_y
date,,
2018-12-01,4214.67,3997.26
2018-12-02,4139.88,3982.33
2018-12-03,3894.13,4212.72
2018-12-04,3956.89,4014.75
2018-12-05,3753.99,4082.48


In [0]:
val_data['diff'] = abs(val_data.close_x - val_data.close_y) / val_data.close_x

In [112]:
val_data.head()

,close_x,close_y,diff
date,,,
2018-12-01,4214.67,3997.26,0.051584
2018-12-02,4139.88,3982.33,0.038057
2018-12-03,3894.13,4212.72,0.081813
2018-12-04,3956.89,4014.75,0.014623
2018-12-05,3753.99,4082.48,0.087504


In [113]:
stats = val_data.mean()
score = 1-stats['diff']
score

0.9330112366227766

**Download Submission File**<br>
Only if the score is at least 90%

In [0]:
if score > 0.90:
  y_sub_df.to_csv(SUB_FILE_NAME)